In [ ]:
# Installing dependencies - LAB 9  COMO BASEE!!!
%pip install tensorflow
%pip install -U 'flwr[simulation]'
%pip install -q "flwr-datasets[vision]"
%pip install torch torchvision
%pip install scikit-learn
%pip install numpy==1.24.3
%pip install Pillow


Note: you may need to restart the kernel to use updated packages.
  Using cached cryptography-44.0.3-cp39-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
INFO: pip is looking at multiple versions of grpcio-health-checking to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_health_checking-1.75.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached grpcio_health_checking-1.75.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached grpcio_health_checking-1.74.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached grpcio_health_checking-1.73.1-py3-none-any.whl.metadata (1.0 kB)
  Using cached grpcio_health_checking-1.73.0-py3-none-any.whl.metadata (1.0 kB)
  Using cached grpcio_health_checking-1.72.2-py3-none-any.whl.metadata (1.0 kB)
  Using cached grpcio_health_checking-1.72.1-py3-none-any.whl.metadata (1.0 kB)
INFO: pip is still looking at multiple versions of grpcio-health-checking to determine which version is compatible with other 

In [12]:
import tensorflow as tf
import flwr as fl
import numpy as np

from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner, DirichletPartitioner
from flwr_datasets.visualization import plot_label_distributions, plot_comparison_label_distribution

from flwr.common import parameters_to_ndarrays, ndarrays_to_parameters
from flwr.common import FitIns, EvaluateIns
from flwr.server.strategy.aggregate import aggregate
import random


class Servidor(fl.server.strategy.FedAvg):
    def __init__(self, num_clients, dirichlet_alpha, fraction_fit=0.2):
        self.num_clients     = num_clients
        self.dirichlet_alpha = dirichlet_alpha

        super().__init__(fraction_fit=fraction_fit, min_available_clients=num_clients)

    def configure_fit(self, server_round, parameters, client_manager):
      """Configure the next round of training."""

      config = {
          'server_round': server_round,
      }
      fit_ins = FitIns(parameters, config)


      sample_size, min_num_clients = self.num_fit_clients(
          client_manager.num_available()
      )
      clients = client_manager.sample(
          num_clients=sample_size, min_num_clients=min_num_clients
      )

      # Return client/config pairs
      print(clients)
      return [(client, fit_ins) for client in clients]

    def aggregate_fit(self, server_round, results, failures):
      parameters_list = []
      
      for _, fit_res in results:
          print(f"Rodada {server_round} resultados de treinamento: {fit_res}")
          parameters = parameters_to_ndarrays(fit_res.parameters)
          exemplos   = int(fit_res.num_examples)

          parameters_list.append([parameters, exemplos])

      agg_parameters = aggregate(parameters_list)
      agg_parameters = ndarrays_to_parameters(agg_parameters)

      return agg_parameters, {}

    def configure_evaluate(self, server_round, parameters, client_manager):
      config = {
          'server_round': server_round,
      }

      evaluate_ins = EvaluateIns(parameters, config)


      sample_size, min_num_clients = self.num_evaluation_clients(
          client_manager.num_available()
      )

      clients = client_manager.sample(
          num_clients=sample_size, min_num_clients=min_num_clients
      )

      return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(self, server_round, results, failures):
      accuracies = []

      for _, response in results:
          acc = response.metrics['accuracy']
          accuracies.append(acc)

      avg_acc = sum(accuracies)/len(accuracies)
      print(f"Rodada {server_round} acurácia agregada: {avg_acc}")

      return avg_acc, {}

In [13]:
import tensorflow as tf
import flwr as fl

import warnings
warnings.filterwarnings("ignore")

def create_client(cid):
    client = Cliente(cid, NIID, NCLIENTS, DIRICHLET_ALPHA, ATTACK_TYPE, FILENAME)
    return client.to_client()


class Simulation():
    def __init__(self):
        self.server  = Servidor(num_clients=NCLIENTS, dirichlet_alpha=DIRICHLET_ALPHA, fraction_fit=FRACTION_FIT)

    def run_simulation(self):
        fl.simulation.start_simulation(
            client_fn        = create_client,
            num_clients      = NCLIENTS,
            config           = fl.server.ServerConfig(num_rounds=NROUNDS),
            strategy         = self.server,
            client_resources = {"num_cpus": 12, "num_gpus": 0.0},)

In [14]:
import plot_utils
plot_utils.plot_loss_acc("model_poisoning")
plot_utils.plot_performance_distribution("model_poisoning", NCLIENTS)

FileNotFoundError: [Errno 2] No such file or directory: 'model_poisoning-train.csv'

In [9]:
def run_ataque(modelo, imagem_fake, label_fake, gradientes_compartilhados,
               optimizer, criterion, ataque='dlg'):
    history     = []
    to_pilimage = transforms.ToPILImage()
    loop = tqdm(range(140), desc=f"🔧 Invertendo Gradientes...",  )
    for iters in loop:
        def closure():
            optimizer.zero_grad()

            pred_ataque = modelo(imagem_fake)

            if ataque == 'dlg':
                onehot_ataque = F.softmax(label_fake, dim=-1)

            elif ataque == 'idlg':
                onehot_ataque = torch.argmin(torch.sum(gradientes_compartilhados[-2],
                                                       dim=-1), dim=-1).detach().reshape((1,)).requires_grad_(False)

            loss_ataque       = criterion(pred_ataque, onehot_ataque)
            gradientes_ataque = torch.autograd.grad(loss_ataque, modelo.parameters(),
                                                    create_graph=True)
            grad_diff = 0

            for gx, gy in zip(gradientes_ataque, gradientes_compartilhados):
                grad_diff += ((gx - gy) ** 2).sum()

            grad_diff.backward()

            return grad_diff

        optimizer.step(closure)
        current_loss = closure().item()
        loop.set_postfix(loss_inversao=current_loss)  # Atualiza a barra de progresso com a loss
        history.append(to_pilimage(imagem_fake[0].cpu()))

    return history

In [10]:
output_modelo    = modelo(imagem)
criterion        = utils.cross_entropy_for_onehot
logits           = criterion(output_modelo, label)
gradientes       = torch.autograd.grad(logits, modelo.parameters())

gradientes_compartilhados = list((_.detach().clone() for _ in gradientes))



NameError: name 'modelo' is not defined

In [11]:
modelo = utils.cria_modelo('cifar10')
utils.poda_modelo(modelo, 0.4)
output_modelo    = modelo(imagem)
criterion        = utils.cross_entropy_for_onehot
logits           = criterion(output_modelo, label)
gradientes       = torch.autograd.grad(logits, modelo.parameters())

gradientes_compartilhados = list((_.detach().clone() for _ in gradientes))

NameError: name 'utils' is not defined